# [FMA: A Dataset For Music Analysis](https://github.com/mdeff/fma)

## Initial Exploration

### Import libraries and data

In [31]:
%matplotlib inline

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

import utils

sns.set_context("notebook", font_scale=1.5)
plt.rcParams['figure.figsize'] = (17, 5)

In [32]:
tracks_raw = utils.load('data/fma_metadata/tracks.csv')
genres_raw = utils.load('data/fma_metadata/genres.csv')
features_raw = utils.load('data/fma_metadata/features.csv')
echonest_raw = utils.load('data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features_raw.index, tracks_raw.index)
assert echonest_raw.index.isin(tracks_raw.index).all()

tracks_raw.shape, genres_raw.shape, features_raw.shape, echonest_raw.shape

((106574, 52), (163, 4), (106574, 518), (13129, 249))

### Explore tracks and echonest df

In [33]:
tracks_raw.columns

MultiIndex([( 'album',          'comments'),
            ( 'album',      'date_created'),
            ( 'album',     'date_released'),
            ( 'album',          'engineer'),
            ( 'album',         'favorites'),
            ( 'album',                'id'),
            ( 'album',       'information'),
            ( 'album',           'listens'),
            ( 'album',          'producer'),
            ( 'album',              'tags'),
            ( 'album',             'title'),
            ( 'album',            'tracks'),
            ( 'album',              'type'),
            ('artist', 'active_year_begin'),
            ('artist',   'active_year_end'),
            ('artist', 'associated_labels'),
            ('artist',               'bio'),
            ('artist',          'comments'),
            ('artist',      'date_created'),
            ('artist',         'favorites'),
            ('artist',                'id'),
            ('artist',          'latitude'),
          

In [34]:
tracks_raw.info()

# there seem to be lot's of Nan values in different columns. 
# In this case: we're mostly interested in the genre related columns from this dataset. 
# We'll explore the propperties table (echonest) before deleting nan values and decing what columns are useful for our ML.

<class 'pandas.core.frame.DataFrame'>
Index: 106574 entries, 2 to 155320
Data columns (total 52 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   (album, comments)            106574 non-null  int64         
 1   (album, date_created)        103045 non-null  datetime64[ns]
 2   (album, date_released)       70294 non-null   datetime64[ns]
 3   (album, engineer)            15295 non-null   object        
 4   (album, favorites)           106574 non-null  int64         
 5   (album, id)                  106574 non-null  int64         
 6   (album, information)         83149 non-null   category      
 7   (album, listens)             106574 non-null  int64         
 8   (album, producer)            18060 non-null   object        
 9   (album, tags)                106574 non-null  object        
 10  (album, title)               105549 non-null  object        
 11  (album, tracks)              10

In [35]:
tracks_raw.head()

album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []   
5                                           <p></p>    6073      NaN   []   
10                                              NaN   47632      NaN   []   
20        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   

          ...       track                         \
          ... information interest language_code   
track_id  ...                                      
2         ...         NaN     4656            en   
3         ...         NaN     1470            en   
5         ...         NaN     1933            en   
10        ...         NaN    54881            en   
20        ...         NaN      978            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151      NaN   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135      NaN   
20        Attribution-NonCommercial-NoDerivatives (aka M...     361      NaN   

                                                 
         number publisher tags            title  
track_id                                         
2             3       NaN   []             Food  
3             4       NaN   []     Electric Ave  
5             6       NaN   []       This World  
10            1       NaN   []          Freeway  
20            3       NaN   []  Spiritual Level  

[5 rows x 52 columns]

In [36]:
echonest_raw.head()

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                                                        ...  \
                                           metadata                     ...   
         speechiness    tempo   valence  album_date         album_name  ...   
track_id                                                                ...   
2           0.159310  165.922  0.576661         NaN                NaN  ...   
3           0.461818  126.957  0.269240         NaN                NaN  ...   
5           0.124595  100.260  0.621661         NaN                NaN  ...   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker  ...   
134         0.525519  114.290  0.894072         NaN                NaN  ...   

                                                                              \
         temporal_features                                                     
                       214        215       216       217       218      219   
track_id                                                                       
2                -1.992303   6.805694  0.233070  0.192880  0.027455  0.06408   
3                -1.582331   8.889308  0.258464  0.220905  0.081368  0.06413   
5                -2.288358  11.527109  0.256821  0.237820  0.060122  0.06014   
10               -3.662988  21.508228  0.283352  0.267070  0.125704  0.08082   
134              -1.452696   2.356398  0.234686  0.199550  0.149332  0.06440   

                                                     
                                                     
               220       221        222         223  
track_id                                             
2          3.67696   3.61288  13.316690  262.929749  
3          6.08277   6.01864  16.673548  325.581085  
5          5.92649   5.86635  16.013849  356.755737  
10         8.41401   8.33319  21.317064  483.403809  
134       11.26707  11.20267  26.454180  751.147705  

[5 rows x 249 columns]

In [37]:
print(echonest_raw.shape)
echonest_raw.columns

# there seem to be lots of columns on this df. 223 columns refer to temporal features. 
echonest_raw['echonest']['temporal_features']

# We probably don't need this. We'll drop them before merging. 


(13129, 249)


,000,001,002,003,004,005,006,007,008,009,...,214,215,216,217,218,219,220,221,222,223
track_id,,,,,,,,,,,,,,,,,,,,,
2,0.877233,0.588911,0.354243,0.295090,0.298413,0.309430,0.304496,0.334579,0.249495,0.259656,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.676960,3.61288,13.316690,262.929749
3,0.534429,0.537414,0.443299,0.390879,0.344573,0.366448,0.419455,0.747766,0.460901,0.392379,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.082770,6.01864,16.673548,325.581085
5,0.548093,0.720192,0.389257,0.344934,0.361300,0.402543,0.434044,0.388137,0.512487,0.525755,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.926490,5.86635,16.013849,356.755737
10,0.311404,0.711402,0.321914,0.500601,0.250963,0.321316,0.734250,0.325188,0.373012,0.235840,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.414010,8.33319,21.317064,483.403809
134,0.610849,0.569169,0.428494,0.345796,0.376920,0.460590,0.401371,0.449900,0.428946,0.446736,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.267070,11.20267,26.454180,751.147705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124857,0.717013,0.686557,0.411056,0.342718,0.341934,0.482926,0.419219,0.408946,0.393060,0.382778,...,-1.721207,4.686078,0.213789,0.208800,0.007911,0.06395,2.040730,1.97678,8.144532,147.040405
124862,0.673395,0.846995,0.447772,0.425936,0.407817,0.405924,0.290565,0.314019,0.318129,0.310359,...,-0.647897,1.282306,0.214586,0.181860,0.011247,0.06240,0.922360,0.85996,1.794739,6.321268
124863,0.842368,0.719091,0.351503,0.354707,0.314619,0.276266,0.340571,0.342762,0.449963,0.456690,...,-0.771613,1.623510,0.180471,0.128185,0.010103,0.06222,2.251160,2.18894,5.578341,89.180328


In [38]:
echonest_raw.info()
# many columns contain Nan values. Fortunately, they are unlikely the most useful columns for our ML model.
# We'll drop them after merging the data frames. 

<class 'pandas.core.frame.DataFrame'>
Index: 13129 entries, 2 to 124911
Columns: 249 entries, ('echonest', 'audio_features', 'acousticness') to ('echonest', 'temporal_features', '223')
dtypes: float64(244), object(5)
memory usage: 25.0+ MB


### Merge dataframes

In [39]:
#  drop temporal_features columns. 
echonest = echonest_raw.copy()
echonest = echonest['echonest'].drop(columns = 'temporal_features')
tracks = tracks_raw.copy()


# merge tracks and echonest 
tracks_ax = pd.merge(tracks, echonest, left_index=True, right_index=True)
tracks_ax['track'].head()

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
134,256000,0,NaN,2008-11-26 01:43:19,2008-11-26,207,3,Hip-Hop,[21],[21],NaN,1126,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,943,NaN,5,NaN,[],Street Music


In [40]:
tracks_ax.columns

MultiIndex([(          'album',                'comments'),
            (          'album',            'date_created'),
            (          'album',           'date_released'),
            (          'album',                'engineer'),
            (          'album',               'favorites'),
            (          'album',                      'id'),
            (          'album',             'information'),
            (          'album',                 'listens'),
            (          'album',                'producer'),
            (          'album',                    'tags'),
            (          'album',                   'title'),
            (          'album',                  'tracks'),
            (          'album',                    'type'),
            (         'artist',       'active_year_begin'),
            (         'artist',         'active_year_end'),
            (         'artist',       'associated_labels'),
            (         'artist',         

In [41]:
# we want to flatten the df to work more confortably and merge more info from the genre csv. 
tracks_ax.columns =  tracks_ax.columns.get_level_values(0) + "_" + tracks_ax.columns.get_level_values(1)
tracks_ax.columns = tracks_ax.columns.str.replace('audio_features_', "").str.replace('social_features_', "").str.replace('metadata_', "md_").str.replace('ranks_', "").str.replace('track_genre', "genre")
tracks_ax.columns

Index(['album_comments', 'album_date_created', 'album_date_released',
       'album_engineer', 'album_favorites', 'album_id', 'album_information',
       'album_listens', 'album_producer', 'album_tags', 'album_title',
       'album_tracks', 'album_type', 'artist_active_year_begin',
       'artist_active_year_end', 'artist_associated_labels', 'artist_bio',
       'artist_comments', 'artist_date_created', 'artist_favorites',
       'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_members', 'artist_name', 'artist_related_projects',
       'artist_tags', 'artist_website', 'artist_wikipedia_page', 'set_split',
       'set_subset', 'track_bit_rate', 'track_comments', 'track_composer',
       'track_date_created', 'track_date_recorded', 'track_duration',
       'track_favorites', 'genre_top', 'genres', 'genres_all',
       'track_information', 'track_interest', 'track_language_code',
       'track_license', 'track_listens', 'track_lyricist', 'track_number'

In [43]:
# one of the problems of this df is that "genre_top", which seems the most indicative column for genre, has lots of NaN values.
# We want to find out if the missing info can be obtained from the genres of genres_all columns. 
# We'll compare the values of these columns. 

# first we get the genre_id of genre_top. 
tracks_ax = pd.merge(tracks_ax, genres_raw.reset_index()[['genre_id', 'title']], left_on = ('genre_top'), right_on = ('title'), how='left')

# genre_id column to top_genre_id
tracks_ax.columns = tracks_ax.columns.str.replace('genre_id', 'genre_top_id')
#tracks_ax['track_genre_test'] = tracks_ax['track_genres_all'].str[0]


In [26]:
# let's create a column that says if the top_genre is nan or not.
tracks_ax['genre_top_isna'] = tracks_ax['genre_top'].isna()

# filter values where the genre_top_id is the same as the first element of genres_all
tracks_ax[tracks_ax['genre_top_id'] == tracks_ax['genres_all'].str[-1]].shape

# this seems to be the best concidence. 
# we'll use it to fill in the nan values of the genre_top column.

tracks_ax['genre_top_id'].fillna(tracks_ax['genres_all'].str[-1], inplace = True)
# there are no null values now. 
tracks_ax['genre_top_id'].isna().sum()

(8063, 79)

np.int64(0)

In [82]:
tracks_ax[(tracks_ax['track_genre_top'] != tracks_ax['title']) & ~(tracks_ax['track_genre_top'].isna())][['track_genre_top', 'title', 'track_genres_all', 'track_genres']]

,track_genre_top,title,track_genres_all,track_genres
track_id,,,,
153,Rock,Post-Rock,"[26, 12]",[26]
154,Rock,Post-Rock,"[26, 12]",[26]
155,Rock,Post-Rock,"[26, 12]",[26]
169,Rock,Punk,"[25, 12]",[25]
170,Rock,Punk,"[25, 12]",[25]
...,...,...,...,...
123821,Rock,Post-Rock,"[26, 12]",[26]
123822,Rock,Post-Rock,"[26, 12]",[26]
123823,Rock,Post-Rock,"[26, 12]",[26]


## Second cleaning

Our goal is to build a ML model that predicts a song genre from it's main audio characteristics (energy, acousticness etc)

In [ ]:
# we find again many columns with Nan values. Most of them are not useful for our ML model. But the genre column is essential. 
# We'll fill in the missing values of genre_top column, with the other genre related columns. 

tracks_ax[tracks_ax['track']['genre_top'].isna()]['track'][['genre_top', 'genres', 'genres_all']].head(50)

In [129]:
genres_raw.columns

Index(['#tracks', 'parent', 'title', 'top_level'], dtype='object')

In [150]:
# let's now get the genre from the genres csv. 
# we will merge tracks_ax and genres_raw based on the genre_id. 
# we will use the genre_test column from tracks_ax as the genre_id column.

genres = pd.DataFrame()
genres['track','genre'] = genres_raw['title']
tracks_ax = pd.merge(tracks_ax, genres, left_on = ('track', 'genre_test'), right_on = ('track', 'genre_id'))

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

In [60]:
test = tracks_ax.copy()

Index(['album_comments', 'album_date_created', 'album_date_released',
       'album_engineer', 'album_favorites', 'album_id', 'album_information',
       'album_listens', 'album_producer', 'album_tags', 'album_title',
       'album_tracks', 'album_type', 'artist_active_year_begin',
       'artist_active_year_end', 'artist_associated_labels', 'artist_bio',
       'artist_comments', 'artist_date_created', 'artist_favorites',
       'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_members', 'artist_name', 'artist_related_projects',
       'artist_tags', 'artist_website', 'artist_wikipedia_page', 'set_split',
       'set_subset', 'track_bit_rate', 'track_comments', 'track_composer',
       'track_date_created', 'track_date_recorded', 'track_duration',
       'track_favorites', 'track_genre_top', 'track_genres',
       'track_genres_all', 'track_information', 'track_interest',
       'track_language_code', 'track_license', 'track_listens',
       'track_

In [135]:
genres.head()

,"(track, genre)"
genre_id,
1,Avant-Garde
2,International
3,Blues
4,Jazz
5,Classical


In [ ]:
# keep droping columns: 
tracks_ax.dropna(columns = ['album', 'comments', 'date_created', 'date_released', 'engineer', 'favorites', 'id'], inplace = True)

In [34]:
# how many not-nan values are in the genre_top columns?
tracks_ax['track']['genre_top'].count()

np.int64(9355)